# Lab 5: Emergence of Reasoning Capabilities of LLMs through Post-Training


In this Lab, we will focus on implementing "from scratch" the following Policy Gradient Algorithms:

- REINFORCE
- Proximal-Policy-Optimization
- Group-Relative-Policy-Optimization

We will be useing the `peft` library and `Low Rank Adpatation` (LORA) to train relatively large models on the `GSM8K` dataset.

In the end, we'll compare our implementation to normalized training with the `trl` library (Transformer Reinforcement Learning)

# Setup

Make sure you use a **GPU Runtime** so that you have access to Collab GPUs.

Follow these steps:

- Click on "Runtime"
- "Change runetime type"
- Select "T4 GPU"
- Restart Runtime

Verify you have a 15360MiB Tesla T4 GPU with the following command

In [2]:
!nvidia-smi

Sun Feb  1 22:24:46 2026       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               Off | 00000000:3B:00.0 Off |                  Off |
| 30%   43C    P8              10W / 230W |      3MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Dataset

GSM8K ([*Training Verifiers to Solve Math Word Problems*](https://arxiv.org/pdf/2110.14168)) is a dataset featuring Grade-School maths word problems.
Each data point is $(question, solution, answer)$ tuple where the solutions are annotated with *reasoning steps and calcualtions* to arrive at the *final answer*. Below are some facts on GSM8K:

- These problems take between 2 and 8 steps to solve.

- Solutions primarily involve performing a sequence of elementary calculations using basic arithmetic operations (+ − ×÷) to reach the final answer.

- A bright middle school student should be able to solve every problem

- Solutions are provided in natural language, as opposed to pure math expressions.


We will pull the dataset from Huggingface

In [31]:
from datasets import load_dataset

dataset = load_dataset('openai/gsm8k', 'main')

In [3]:
# GSM8K comes two splits: `train` and `test`
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})


In [4]:
import random

train_dataset = dataset['train']
test_dataset = dataset['test']

num_train = len(train_dataset)
num_test = len(test_dataset)

# picking a random example from train test
sample_idx = random.randint(0,num_train-1)
sample = train_dataset[sample_idx]

print(f'Question:\n\n{sample["question"]}\n\n' + '-'*50 + f'\n\nAnswer\n\n{sample["answer"]}')

Question:

Colton had 72 dolphin stickers. He gave 4 stickers each to 3 friends.  He also gave his friend Mandy 2 more than he gave his three friends total.   And he gave Justin 10 less than Mandy.  How many stickers does Colton have left?

--------------------------------------------------

Answer

Colton gave away 4 stickers * 3 friends = <<4*3=12>>12 stickers.
Colton gave Mandy 2 + 12 stickers = <<2+12=14>>14 stickers.
Colton gave Justin 14 stickers - 10 = <<14-10=4>>4 stickers.
In total, Colton gave away 12 + 14 + 4 stickers = <<12+14+4=30>>30 stickers.
Colton's number of total stickers at the end is 72 - 30 = <<72-30=42>>42 stickers.
#### 42


In this Lab, we'll use the first 8 training samples at shots for the CoT prompt we'll define below

In [5]:
cot_dataset = dataset['train'].select(range(8))
train_dataset = dataset['train'].select(range(8, len(dataset['train'])))

In [6]:
import re
INVALID_ANSWER = '[invalid]'

def extract_answer(text) :
  match = re.search("\n#### ([-]?[0-9.,]+)", text)
  # handle cases where the regex match returns nothing
  if match :
        answer = match.group(1).strip()
        return re.sub('[.,!;]', '', answer)
  else :
    return INVALID_ANSWER

def extract_reasoning(text) :
  # replacing the reasoning part
  reasoning = re.sub('<<[^>]+>>', '' ,text)
  # replacing the answer part
  return re.sub("\n#### ([-]?[0-9.,]+)", '' , reasoning)



## Chain-of-Thoughts Prompting

As done usually, we will be using CoT prompting to generate rollouts. We use CoT to force the trained policy (model) to generate reasoning steps before answering

We can use the functions we built in Lab 4

In [7]:
# we define a set of global variables that we will use throughout the lab

ANSWER_INSERT_STR = "The answer is "
QUESTION_STR = "Q: "
ANSWER_STR = "A: "
INSTRUCTIONS= "Solve the following math problem. Follow the example format.\n\n"

In [8]:
import numpy as np

def get_cot_shots(train_dataset, n_shots=8) :
  # Fixing numpy seed to get deterministic results
  np.random.seed(2026)

  # sample 8 examples from the dataset
  indices = np.arange(len(train_dataset))
  np.random.shuffle(indices)
  shots_idx = indices[:n_shots]

  # iteratively build the examples
  few_shot_prompt = INSTRUCTIONS
  for idx in shots_idx :
    sample = train_dataset[int(idx)]
    question, answer = sample['question'], sample['answer']
    solution = extract_answer(answer)
    reasoning = extract_reasoning(answer)
    few_shot_prompt+= QUESTION_STR + question + "\n" +\
                      ANSWER_STR  + reasoning + " " + ANSWER_INSERT_STR + solution + "\n"
  return few_shot_prompt

def build_cot_few_shot_prompt(train_dataset, test_dataset, test_index, n_shots=8) :
  # getting the examples
  shots = get_cot_shots(train_dataset, n_shots=n_shots)

  # building the final prompt
  sample = test_dataset[test_index]
  question, answer = sample['question'], sample['answer']
  solution = extract_answer(answer)
  prompt = shots + QUESTION_STR + question + "\n" + ANSWER_STR.strip()
  return prompt, solution

INVALID_ANSWER = '[invalid]'

def extract_answer_from_model(text) :
    text = text.replace('\\boxed','')
    text = re.sub('[\*\$\{\}]', '', text)
    match = re.search("The answer is ([- ]?[0-9., ]+)", text)
    # handle cases where the regex match returns nothing
    if match :
        answer = match.group(1).strip()
        return re.sub('[.,!;]', '', answer)
    else :
        return INVALID_ANSWER

# Model Instanciation

Below we define the generation config we'll use throughout this example

## Setting up the QLORA model

LoRA is an effective way of fine-tuning LLMs. Very briefly, it introduces **learnable parameters** on top of the **initial, frozen model parameters**. These learnable parameters are called **adaptors**, and only these parameters will be changed during training.

In addition, QLoRA applies a **quantization** to the model parameters: it effectively reduce their memory footprint by *quantizing* their value (*e.g.* $4.161234$ --> $4.16$).

While QLoRA can be worse than *full fine-tuning* of the model parameters, it allows us in this Lab to use **Large Models** that are already good at following instructions. In other words, we are using a "warmed-up" model before our Reinforcement Learning training, which is desirable as we saw in the Lecture.

I think you can try first without quantization, and use quantization if you run into GPU VRAM problems

In [9]:
from peft import LoftQConfig, LoraConfig, get_peft_model, TaskType
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
# from bits

# Configure Quantization 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)


# model_name = "Qwen/Qwen3-0.6B"
model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16,
    # quantization_config=bnb_config,
    # device_map="cuda:0",
)

# Configure LoRA
rank = 16
peft_config = LoraConfig(
    r=rank,
    lora_alpha=rank*2,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    task_type="CAUSAL_LM",
    bias='none',
    lora_dropout=0.05,
)




Loading weights: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 311/311 [00:00<00:00, 1387.99it/s, Materializing param=model.norm.weight]
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


## Setting up the (Q)LoRA Model for SFT

In [10]:
# # Apply PEFT model adaptation
lora_model = get_peft_model(model, peft_config)

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
print_trainable_parameters(lora_model)
generation_config = GenerationConfig.from_dict(
    {
        "bos_token_id": 151643,
        "do_sample": True,
        "eos_token_id": [
            151645,
            151643
        ],
        "pad_token_id": 151643,
        "temperature": 1.0,
        "max_new_tokens" : 250, 
        "top_k": 20,
        "top_p": 0.95
    }
)

trainable params: 10092544 || all params: 761724928 || trainable%: 1.32


In [11]:
import torch
device = torch.device('cuda:0')
lora_model = lora_model.to(device)

### Setting up the Reference Model to test Base Model

In [12]:
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16,
    device_map="cpu",
)
ref_model = ref_model.to(device)

Loading weights: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 311/311 [00:00<00:00, 1253.36it/s, Materializing param=model.norm.weight]
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


## Tokenization Utils

Note that Qwen3 models have already been RL post-trained to generate reasoning steps in between `<think>` and `</think>` tokens. We can disable this by default in the tokenizer call.

We want to disable it as our goal is to induce this behavior as well

In [13]:
## Setting up some utility function that will tokenize and detokenize inputs 

def tokenize_inputs(prompt, tokenize_only=False) : 
    messages = [{'role' : 'user', 'content'  :prompt}]
    if not tokenize_only : 
        return tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=True, enable_thinking=False)
    else : 
        return tokenizer(prompt, return_tensors='pt', add_generation_prompt=False, enable_thinking=False)


def detokenize_out(prompt_ids, response) :
    answer_ids = [response[i, prompt_ids[i].size(0):] for i in range(len(prompt_ids))]
    out_texts = tokenizer.decode(answer_ids)
    return out_texts

## Batching

Instead of generating samples one-by-one which can be really slow, we will batch our inputs. Batching implies that all inputs have the same size --> we use a strategy called `padding` to make all questions of different length have the same sequence length

In [14]:
from torch.nn.utils.rnn import pad_sequence
from typing import List ,Dict

def DataCollator(inputs: List[str], use_labels=False) -> Dict[str,torch.Tensor] : 

    input_dicts = [tokenize_inputs(i['prompt'], tokenize_only=False) for i in inputs]

    input_ids = [i['input_ids'].squeeze() for i in input_dicts]
    mask = [i['attention_mask'].squeeze() for i in input_dicts]

    # for SFT, we'll use labels which are just the actual Token IDs inside the prompt + answer
    if use_labels : 
        labels =  [tokenize_inputs(i['answer'], tokenize_only=True)['input_ids'].squeeze() for i in inputs]
        labels_pad = [torch.full_like(i, -100) for i in input_ids]
        input_ids = [torch.cat((i, l )) for i, l in zip(input_ids, labels)]
        labels = [torch.cat((lpad, l )) for lpad, l in zip(labels_pad, labels)]
        
    ### PADDING ###
    
    # 151643 is the token ID of the pad token in the tokenizer, but we can find it dynamically
    pad_token = tokenizer.special_tokens_map['pad_token']
    pad_token_id = tokenizer.convert_tokens_to_ids(pad_token)
    
    # Here we peform padding from the left side --> [This is Lab 5 ] --> [PAD PAD PAD This is Lab 5]
    # for Input Ids, we replace PAD with the Pad token ID
    input_ids = pad_sequence(input_ids, batch_first=True, padding_side='left', padding_value = pad_token_id)
    # for Attention Masks, we replace PAD with a value of 0 to ensure that self-attention does not caputre PAD tokens
    mask = pad_sequence(mask, batch_first=True, padding_side='left', padding_value = 0)
    if use_labels : 
        # By convention, the label values of PAD tokens are set to -100
        labels = pad_sequence(labels, batch_first=True, padding_side='left', padding_value = -100)

        return {'input_ids' :input_ids, 'attention_mask' : mask, 'labels':labels}
    else : 
        return {'input_ids' :input_ids, 'attention_mask' : mask}

## Check if everything is okay

In [16]:
q, a = build_cot_few_shot_prompt(cot_dataset, test_dataset, 10, 8)

print(f'The answer is {a}.')

The answer is 366.


In [16]:
inputs  = tokenize_inputs(q)

In [18]:
print(tokenizer.decode(inputs['input_ids'][0]))

<|im_start|>user
Solve the following math problem. Follow the example format.

Q: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?
A: He writes each friend 3*2=6 pages a week
So he writes 6*2=12 pages every week
That means he writes 12*52=624 pages a year The answer is 624
Q: Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?
A: There are 80/100 * 10 = 8 more purple flowers than yellow flowers.
So in Mark's garden, there are 10 + 8 = 18 purple flowers.
Purple and yellow flowers sum up to 10 + 18 = 28 flowers.
That means in Mark's garden there are 25/100 * 28 = 7 green flowers.
So in total Mark has 28 + 7 = 35 plants in his garden. The answer is 35
Q: Julie is reading a 120-page book. Yesterday, she was able t

In [19]:
lora_model.eval()
out = lora_model.generate(
    **{k:v.to(device) for k,v in inputs.items()},
    generation_config=generation_config
)

In [20]:
print(detokenize_out(inputs['input_ids'],out)[0])

A: 60 + (60 * 3) - (60 * 30% of 60) = 60 + 180 - 108 = 132 downloads  
The answer is 132<|im_end|>


# Base Model Performance on a Small Subset

In [21]:
NUM_SUBSET = 100

torch.manual_seed(42)
random_indices = torch.randperm(len(test_dataset))[:NUM_SUBSET]
random_indices[:10]

tensor([ 970,   58,  813,  645, 1180, 1258,  925,  731,  906,  952])

In [26]:


questions, answers = [], []
for i in random_indices : 
    q, a = build_cot_few_shot_prompt(cot_dataset, test_dataset, int(i), n_shots=8)
    questions.append({'prompt' : q})
    answers.append(a)

## Base Model: Eval 

In [27]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm 

loader = DataLoader(questions, batch_size = 4, shuffle=False, collate_fn = DataCollator)

In [28]:
print(next(iter(loader)))

{'input_ids': tensor([[151643, 151643, 151643,  ...,    271, 151668,    271],
        [151643, 151643, 151644,  ...,    271, 151668,    271],
        [151644,    872,    198,  ...,    271, 151668,    271],
        [151643, 151643, 151643,  ...,    271, 151668,    271]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])}


### Excerise

Loop over the data loader to generate completions with the **Reference Model** and get the resulting text

In [38]:
decoded_outs = []

# making sure that dropout are disabled
lora_model.eval()

# we don't calculate gradients: we just perform inference
with torch.no_grad() : 
    for batch in tqdm(loader) : 
        # putting the batch on the right GPU
        batch = "..."
        # Generate completions for this batch with the `generate` function
        # Do not forget to use the generation config
        out = "...'

        # Detokenize the output ids back into text
        # Hint: use one of the Tokenization function from above
        decoded_text = "..."
        decoded_outs.append(decoded_text)

In [36]:
# # Flatten list of list into list
decoded_outs = sum(decoded_outs, [])

In [34]:
len(decoded_outs)

100

In [35]:
pred_answers = [extract_answer_from_model(gen) for gen in decoded_outs]
accuracy = sum([(p==gt) for p,gt in zip(pred_answers, answers)]) / len(answers)
print(f"(Base Model) Accuracy on Test Subsample: {int(accuracy*100)}%")

(Base Model) Accuracy on Test Subsample: 31%


Note: your scores might differs because of sampling parameters

# Supervised-Fine-Tuning (SFT)

For SFT, we also need labels. For Language Modeling, labels are simply the next token to be predicted. By convention, we can simply use the answer Token IDs as labels, and the `transformers` model will take care of calculating the language modeling loss

We'll also use 8 shots during SFT. You can reduce this to reduce memory constraints

In [47]:
NUM_TRAIN = 500

torch.manual_seed(42)
random_indices = torch.randperm(len(test_dataset))[:NUM_TRAIN]

questions, answers = [], []
for i in random_indices : 
    # Reduce the number of shots if you get an OOM error
    q, a = build_cot_few_shot_prompt(cot_dataset, train_dataset, int(i), n_shots=0)
    # we also need the actual answer: 
    reasoning = extract_reasoning(train_dataset[int(i)]['answer'])
    # here we add <|im_end|> at the end as it is the end of sentence token
    questions.append({'prompt' : q, 'answer' : reasoning + ' ' + f'The answer is {a}<|im_end|>' })
    answers.append(a)

In [48]:
print(len(questions))

500


In [49]:
from torch.utils.data import DataLoader
from functools import partial

collate_fn = partial(DataCollator, use_labels=True)

loader = DataLoader(questions, batch_size=8, collate_fn=collate_fn)

In [29]:
batch = next(iter(loader) )
print(tokenizer.decode(batch['input_ids'])[0].replace('<|im_end|>', '').replace('<|endoftext|>', ''))

<|im_start|>user
Solve the following math problem. Follow the example format.

Q: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?
A: He writes each friend 3*2=6 pages a week
So he writes 6*2=12 pages every week
That means he writes 12*52=624 pages a year The answer is 624
Q: Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?
A: There are 80/100 * 10 = 8 more purple flowers than yellow flowers.
So in Mark's garden, there are 10 + 8 = 18 purple flowers.
Purple and yellow flowers sum up to 10 + 18 = 28 flowers.
That means in Mark's garden there are 25/100 * 28 = 7 green flowers.
So in total Mark has 28 + 7 = 35 plants in his garden. The answer is 35
Q: Zeke's baseball team has 7 more players than Carlton's. If 

### Exercise

Complete the SFT Loop

In [51]:
from tqdm.auto import tqdm

def SFTLoop(
    model,
    loader,
    num_epochs=3,
    lr=2e-5
) : 
    # build the optimizer with the learning rate and model parameters
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    
    for e in range(num_epochs) : 
        for batch in tqdm(loader) : 
            # Put data on the right device
            batch = "..."
            # Forward pass on the model --> what does it returns?
            model_out = model(**batch)
            # By default the ModelForCausalLM class returns the language modeling loss if provided with labels
            loss = "..."

            # zero-out-gradient
            optimizer.zero_grad()
            # calculating gradients
            loss.backward()
            # Here we clip the gradients to avoid too large updates
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # Then we perform the gradient descent step
            optimizer.step()
    return model

In [55]:
trained_lora = SFTLoop(lora_model, loader)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:20<00:00,  3.14it/s]


In [56]:
trained_lora.save_pretrained('./sft_lora_500sample_0shots_2epochs.pt')

In [38]:
from peft import PeftModel

trained_lora = PeftModel.from_pretrained(model, './sft_lora_800sample.pt/')

/usr/local/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [39]:
trained_lora = trained_lora.to(device)

## Evaluation

In [57]:
NUM_SUBSET = 100

torch.manual_seed(42)
random_indices = torch.randperm(len(test_dataset))[:NUM_SUBSET]

questions, answers = [], []
for i in random_indices : 
    q, a = build_cot_few_shot_prompt(cot_dataset, test_dataset, int(i), n_shots=8)
    questions.append({'prompt' : q})
    answers.append(a)

In [58]:
loader = DataLoader(questions, batch_size = 4, shuffle=False, collate_fn = DataCollator)

### Exercise

Build the same inference loop as above, but with the trained model this time

In [ ]:
decoded_outs = []

# making sure that dropout are disabled
trained_lora.eval()

with torch.no_grad() : 
    for batch in tqdm(loader) : 
        # putting the batch on the right GPU
        batch = "..."
        # Generate completions for this batch with the `generate` function
        # Do not forget to use the generation config
        out = "...'

        # Detokenize the output ids back into text
        # Hint: use one of the Tokenization function from above
        decoded_text = "..."
        decoded_outs.append(decoded_text)

In [67]:
# Flatten list of list into list
decoded_outs = sum(decoded_outs, [])
pred_answers = [extract_answer_from_model(gen) for gen in decoded_outs]
accuracy = sum([(p==gt) for p,gt in zip(pred_answers, answers)]) / len(answers)
print(f"(SFT Model) Accuracy on Test Subsample: {int(accuracy*100)}%")

(SFT Model) Accuracy on Test Subsample: 6%


- **(Base Model) Accuracy on Test Subsample: 31%**
- **(SFT Model) Accuracy on Test Subsample: 6%**

We see a reduction after training for a few training iterations, as it is often the case when SFT on reasoning data

In [69]:
invalids = [i for i in pred_answers if 'invalid' in i]
print(f'% of Invalid Answers: {(len(invalids)/len(decoded_outs)) * 100:.2f}')

% of Invalid Answers: 2.00


Look at one generated answer

In [72]:
print(decoded_outs[0].replace('<|im_end|>', '').replace('<|endoftext|>',''))

Carlton's team has 7 more players than Zeke.
Carlton's team has 13+7=20 players in both teams.
The answer is 20
The answer is 20
The answer is 20


# Policy Gradient Algorithms

In this part, we'll implement GRPO with the same training data, and compare its performance to the Base and SFT model.

We'll start by building a base trainer class. This class should have all necessary methods that are needed for RL training, no matter the Policy Gradient Optimization algorithm used

**Note that you should restart the Ipython Kernel at this point to load a fresh LoRA Model**

### Some logging utilities

In [ ]:
import logging
import sys 
from pathlib import Path 
import re 

def setup_logger(
    name: str = "notebook",
    log_file: str = "notebook.log",
    level: int = logging.INFO,
):
    """Just a simple function that creates a logger that will send output to the console + a file"""
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.propagate = False  # prevent double logging via root

    # Clear existing handlers (important in Jupyter!)
    if logger.handlers:
        logger.handlers.clear()

    formatter = logging.Formatter(
        "%(asctime)s | %(levelname)-8s | %(name)s | %(message)s",
        datefmt="%H:%M:%S",
    )

    # --- Notebook output handler ---
    stream_handler = logging.StreamHandler(sys.stdout)
    stream_handler.setLevel(level)
    stream_handler.setFormatter(formatter)

    # --- File handler ---
    Path(log_file).parent.mkdir(parents=True, exist_ok=True)
    file_handler = logging.FileHandler(log_file, mode="a")
    file_handler.setLevel(level)
    file_handler.setFormatter(formatter)

    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)

    return logger

    

### Exercise

Complete the following functions:

- `get_policy_completions`
- `get_logprobs`
- `format_reward_fn`
- `correct_reward_fn`

In [17]:

class LoRA_RLTrainer() : 
    def __init__(
        self,
        lora_model,
        ref_model,
        tokenizer,
        generation_config,
        device,
        optimizer = 'adamw',
        num_rollouts=2,
        group_size=1,
        lr=2e-5,
        beta=0.05,
        gamma = 1.0,
        use_returns=True,
        kl_in_reward=True,
        gradient_accumulation_steps=1,
        num_policy_steps=1,
        cliprange=0.2
    ):
        """Base class for training a model with Policy Gradient Optimization algorithms"""
        self.model = lora_model
        self.ref_model = ref_model
        
        # No gradient in reference model 
        for p in self.ref_model.parameters() : 
            p.requires_grad = False 
            
        self.tokenizer = tokenizer
        self.device=device
        self.num_rollout = num_rollouts
        self.group_size = group_size
        self.generation_config = generation_config
        self.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.special_tokens_map['eos_token'])
        self.beta = beta
        self.gamma = gamma
        self.use_returns = use_returns
        self.kl_in_reward = kl_in_reward
        self.gradient_accumulation_steps = gradient_accumulation_steps
        if optimizer == 'adamw' : 
            self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=lr)
        self.logger = setup_logger()
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.num_policy_steps = num_policy_steps
        self.cliprange = cliprange
        
    
    def log(self, step, max_step, loss, mean_reward, kl_div, length) : 
        """Log a training step"""
        log_text = f"[{step+1}/{max_step}] Loss: {loss:0.3f} | Mean Reward: {mean_reward:0.3f} | KL: {kl_div:0.3f} | Mean Length: {length:0.1f}"
        self.logger.info(log_text)
        
    def mask(self, token_ids) : 
        """Given token ids, returns a mask where all position with a pad tokens are set to zero"""
        mask = torch.ones_like(token_ids)
        mask[torch.where(token_ids == 151643)] = 0
        return mask 

    def expand_batch(self, batch, num_sequences) :
        """Repeats a batch of shape [B,S] to [B*num_sequences, S]"""
        if num_sequences > 1 : 
            return  {
            k : v.repeat_interleave(num_sequences, dim=0)# [B * G, S]
            for k,v in batch.items()
        }
        else : 
            return batch

    @torch.no_grad()
    def get_policy_completions(self, prompts) : 
        """Generate completions to prompts with the policy model, and returns both the prompts and completion IDs"""
        batch = DataCollator(prompts)
        B = batch['input_ids'].size(0)

        # expand the batch to [B*num_sequences, S]
        batch = "..."

        # generate B*num_sequences completions
        out = self.model.generate(**{k:v.to(device) for k,v in batch.items()}, generation_config=generation_config)

        # Resize to [B, num_sequences, S]
        out = "..."
        # Divide between prompts and complteions
        completions = out[:,:,"..."]
        prompts =  out[:,:, "...")]
        
        return prompts, completions
    
    def get_logprobs(self, model, prompts, completions) : 
        """Perform a forward pass on `model` with prompts and compltions, and return completion log probabilities"""
        S = prompts.size(2)

        # concatenate prompts and completions
        inputs = "..."
        B,G,T = inputs.size()
        # Resize 
        inputs = inputs.view(B*G,T)
        # Get padding mask
        mask = self.mask(inputs)

        # forward pass
        out = model(inputs.to(device), attention_mask=mask.to(device))

        # Get log probabilities -->  log(softmax(logits))
        logits = "..."
        logprobs = "..." 
        
        # Get completions IDs
        completion_ids = inputs[:,S:]

        # Get log-probabilities for completion only. Should be [BG, T]
        token_log_probs = torch.gather("...").squeeze(-1)
        
        BG, T = token_log_probs.size()
        # Resize
        token_log_probs = token_log_probs.view(B,G,T)
        
        return token_log_probs, mask[:,S:].view(token_log_probs.size())
        
    @torch.no_grad()
    def get_ref_logprobs(self, prompts, completions) : 
        """Simple wrapper to get reference log-probabilities"""
        return self.get_logprobs(self.ref_model, prompts, completions)
        
    def get_policy_logprobs(self, prompts, completions) : 
        """Simple wrapper to get policy log-probabilities"""
        return self.get_logprobs(self.model, prompts, completions)

    def get_kl_from_logprobs(
        self, ref_logp, logp
    ) : 
        """Simple wrapper to get KL Divergence"""
        return torch.exp(ref_logp -  logp) - (ref_logp - logp) - 1

    
    def format_reward_fn(self, completions):
        """A simple reward function that gives a reward when the answer is well-formated."""
        
        pattern = r"..."
        matches = ["..." for content in completion_contents]
        return [0.5 if match else 0.0 for match in matches]

    def correct_reward_fn(self, completions, answers) :
        """An accuracy reward function that gives a reward when the answer and completions are equal"""
        return  [1  if "..." else 0 for i in range(len(completions))]
        
    def reward_fn(self, completions, gt_answers, ref_logp=None, logp=None, mask=None) :
        """Given completions and ground-truth solutions, compute the rewards"""
        
        B, G, T = completions.size()
        pred_ids = completions.view(B*G, T)
        # Get completion in text
        pred_text = self.tokenizer.decode(pred_ids)
        # Extract answer
        pred_answers = [extract_answer_from_model(i) for i in pred_text]
        # Expand answers to have same number of values
        gt_answers = [[a] * G for a in gt_answers] # Expand
        gt_answers = sum(gt_answers, []) # Flatten

        # Calculate rewards
        format_rewards = self.format_reward_fn(pred_text)
        correct_rewards = self.correct_reward_fn(pred_answers, gt_answers)
        reward_per_completion = [correct_rewards[i] + format_rewards[i] for i in range(len(correct_rewards))] 

        # Apply rewards to all tokens 
        rewards = torch.tensor(reward_per_completion, dtype=rewards.dtype).unsqueeze(1)
        
        # create a "Done mask" where value of 1 indicate EOS token
        done_mask = torch.zeros_like(rewards)
        eos_indices = torch.where(pred_ids == self.eos_token_id)
        done_mask[eos_indices] = 1

        # Back to [B,G,T] 
        rewards = rewards.view(B, G, T)
        done_mask = done_mask.view(B, G , T)
        if self.kl_in_reward : 
            # Apply KL inside the rewards if necessary: done in PPO but not in GRPO
            per_token_kl_div = self.get_kl_from_logprobs(ref_logp, logp) * mask
            penalized_rewards = rewards - self.beta * per_token_kl_div
        else : 
            per_token_kl_div = 0
            penalized_rewards = rewards

        # For log only
        mean_rewards = rewards[:, -1].mean()
        return penalized_rewards, mean_rewards, done_mask, per_token_kl_div

    def get_returns(self, completions, gt_answers, ref_logp=None, logp=None, mask=None) : 
        """
        Estimates discounted returns. Note that it won't work exacly as intended for now since we apply 
        the same rewards to all tokens in the `reward_fn` function. Optimally, the reward should be applied to the `EOS` token position only
        before using `get_returns`.
        """
        penalized_rewards, mean_rewards, done_mask, per_token_kl_div = self.reward_fn( completions, gt_answers, ref_logp, logp, mask) 
        
        returns = torch.zeros_like(penalized_rewards)
        running_return = 0.0
        
        for t in reversed(range(returns.size(2))):
            running_return = penalized_rewards[:,:, t] + self.gamma * (1 - done_mask[:, :, t]) * running_return
            returns[:,:, t] = running_return
            
        return returns, mean_rewards, per_token_kl_div

    def generate_rollouts(self, data) :
        """Generates a set of rollout and its statistics"""
        prompts = [d for d in data]
        answers = [d['answer'] for d in data]

        all_prompts, all_preds = self.get_policy_completions(prompts) 
        print([self.tokenizer.decode(all_preds[0,0]).replace("<|im_end|>","").replace('<|endoftext|>', '')])
        print(answers[0])

        # [B, G, T]
        logprobs, mask = self.get_policy_logprobs(all_prompts, all_preds)
        # [B, G, T]
        ref_logprobs,_ = self.get_ref_logprobs(all_prompts, all_preds)

        if self.use_returns : 
            rewards, mean_rewards, per_token_kl_div = self.get_returns(all_preds, answers, ref_logprobs, logprobs, mask)
        else : 
            rewards, mean_rewards, _, per_token_kl_div  = self.reward_fn(all_preds, answers)
            
        return {
            'completions' : all_preds,
            'prompts' : all_prompts,
            'logprobs' : logprobs,
            'ref_logprobs' : ref_logprobs,
            'rewards' : rewards,
            'mean_rewards' : mean_rewards,
            'mask' : mask,
            'per_token_kl_div' : per_token_kl_div
        }
            
    def train_step(self) :      
        """To Implement"""
        pass

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 233)

## GRPO

In [19]:
generation_config = GenerationConfig.from_dict(
    {
        "bos_token_id": 151643,
        "do_sample": True,
        "eos_token_id": [
            151645,
            151643
        ],
        "pad_token_id": 151643,
        "temperature": 1.0,
        "max_new_tokens" : 300, 
        "top_k": 20,
        "top_p": 0.95
    }
)


### Exercise

Implement the `train_step` function as for GRPO

Recall the PPO equation:

$$
J(\theta)
  = \mathbb{E}_{t}[
  \min\left(\frac{\pi_\theta(a\mid q)}{\pi_{\theta_{\text{old}}}(a\mid q)}{A},\
  \text{clip}(\frac{\pi_\theta(a\mid q)}{\pi_{\theta_{\text{old}}}(a\mid q)},1-\varepsilon,1+\varepsilon){A}\right)
$$

In [20]:
from tqdm.auto import tqdm
class GRPOTrainer(LoRA_RLTrainer) : 
    
    def train_step(self, step, data) :      
        """To Implement"""
        # 1. Compute Rollouts
        rollouts = self.generate_rollouts(data)
        
        B,G,T = rollouts['rewards'].size()
        
        # 2.1 Compute advantage shape [B]
        group_mean = "..." 
        group_std = "..." 
        # 2.2 Expand to have [B*G] shape
        group_mean = "..." 
        group_std = "..." 
        # 2.3 calculate advantages
        advantages = (rollouts['rewards'].view(B*G,T) - group_mean)  / (group_std + 1e-4)
        
        # 3. Whitening advantages for stability 
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-4)
        
        # 4. Off policy steps
        prompts = rollouts['prompts'] 
        completions = rollouts['completions']
        # Note that we detach her because the old and reference logprobs are treated as external
        old_per_token_log_probs = rollouts['logprobs'].detach()
        ref_token_log_probs = rollouts['ref_logprobs'].detach()

        # Perform multiple policy steps with mini batches
        for it in range(0, B, self.policy_batch_size) : 
            # build minibatches
            mb_advantages = advantages.view(B,G,T)[it:it+self.policy_batch_size]
            mb_advantages = mb_advantages.view(-1, T)
            mb_ref_logprobs = ref_token_log_probs[it:it+self.policy_batch_size].view(-1, T)
            mb_old_logprobs = old_per_token_log_probs[it:it+self.policy_batch_size].view(-1, T)
            mb_prompts = prompts[it:it+self.policy_batch_size]
            mb_completions = completions[it:it+self.policy_batch_size]
            
            # Get current policy log probabilities
            token_log_probs = "..." 

            # Resize
            token_log_probs = token_log_probs.view(self.policy_batch_size*G, T)

            # calculate PPO ratio: pi_theta / pi_old
            ratio = "..."
            
            eps = self.cliprange  # epsilon in PPO
            pg_losses1 = "..." # unclipped term in PPO (B*G, T)
            pg_losses2 = "..." # clipped term in PPO (B*G, T) hint: look at torch.clamp function
            # PPO loss
            pg_loss_max = -torch.min(pg_losses1, pg_losses2)  # Shape: (B*G, T)       
            
            # KL div
            per_token_kl = self.get_kl_from_logprobs(mb_ref_logprobs, token_log_probs)
            
            # important to GRPO -- PPO applies this in reward traditionally
            # Combine with KL penalty
            per_token_loss = pg_loss_max + self.beta * per_token_kl  # Shape: (B*G, T)
            
            # Get mini batch mask
            mask =  rollouts['mask'][it:it+self.policy_batch_size].view(-1, T)

            # Calculate loss with masking 
            loss = ((per_token_loss * mask).sum(dim=1) / mask.sum(dim=1)).mean()
            
            # Perform gradient step
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.5)  # Stabilize training
            self.optimizer.step()
            
            ## For Logs
            mean_completion_length = mask.sum(-1).float().mean()

        with torch.no_grad() : 
            kl_div = self.beta * (per_token_kl * mask).sum() / mask.sum()
            
        return {
            'loss' : loss,
            'mean_reward' : (rollouts['rewards']*mask.view(-1,G,T)).mean(),
            'kl_div' : kl_div,
            'mean_completion_length' : mean_completion_length
        }
    
    def train(self, dataset) : 
        batches = []
        for bidx in range(0, len(dataset), self.num_rollout) : 
            batches.append(dataset[bidx:bidx+self.num_rollout])

        self.model.train()

        log_in = {
            'loss' : 0 ,
            'mean_reward' : 0, 
            'kl_div' : 0, 
            'mean_completion_length' : 0, 
        }
        
        for step, batch in tqdm(enumerate(batches), total=len(batches)) : 
            # print(batch)
            step_out = self.train_step(step, batch)
            log_in = {k : log_in[k] + step_out[k] for k,v in step_out.items()}
            
            if (step+1) % self.gradient_accumulation_steps == 0 : 
                self.log(
                    step,
                    len(batches),
                    log_in['loss'] / self.gradient_accumulation_steps,
                    log_in['mean_reward'] / self.gradient_accumulation_steps,
                    log_in['kl_div'] / self.gradient_accumulation_steps,
                    log_in['mean_completion_length'] / self.gradient_accumulation_steps
            )
                log_in = {
                    'loss' : 0 ,
                    'mean_reward' : 0, 
                    'kl_div' : 0, 
                    'mean_completion_length' : 0, 
                }
                

In [ ]:
## We will be using only 200 samples here for faster training
NUM_TRAIN = 200

torch.manual_seed(42)
random_indices = torch.randperm(len(test_dataset))[:NUM_TRAIN]


train = []
# We only use 2 shots here for faster training as well
for i in random_indices : 
    q, a = build_cot_few_shot_prompt(cot_dataset, train_dataset, int(i), n_shots=2)
    train.append({'prompt' : q, 'answer' : a})

In [22]:
trainer= GRPOTrainer(
    lora_model,
    tokenizer,
    generation_config,
    device,
    lr=2e-5, # Learning rate. usually very low in GRPO
    cliprange=0.2, # epsilon value in the PPO-Clip objective
    group_size=2, # Size of Group, i.e. the number of generations (a_i) for each prompt q
    num_rollouts=3, # Number of (prompt, answer) pairs generated at once
    policy_batch_size=1, # mini-batch size for off-policy iterations
    use_returns=False, # Whether to use returns rather than rewards
    kl_in_reward=False, # Should kl be computed inside the reward
    beta = 0.05 # KL-Div coefficient. A higher beta means kl is more important
)

trainer.train(train)

  0%|                                                                                                                                                                                | 0/67 [00:00<?, ?it/s]

['Q: Stephen has 110 ants in his ant farm. Half of the ants are worker ants, 20 percent of the worker ants are male. How many female worker ants are there?\n\nA: Half of 110 is 55.  \n20% of 55 is $ \\frac{20}{100} \\times 55 = 11 $.  \nSo there are 11 female worker ants.  \n\nThe answer is 11.']
GT Answer:  44
23:16:25 | INFO     | notebook | [1/67] Loss: 6.069 | Mean Reward: 0.345 | KL: 6.062 | Mean Length: 125.5


  1%|██▌                                                                                                                                                                     | 1/67 [00:12<13:18, 12.10s/it]

['Q: Fred spent half of his allowance going to the movies. He washed the family car and earned 6 dollars. What is his weekly allowance if he ended with 14 dollars?\n\nA: Fred spent half of his allowance, leaving 14 - 6 = 8 dollars. So half of that is $8, which means his weekly allowance was $16.\n\nThe answer is 16.']
GT Answer:  16
23:16:36 | INFO     | notebook | [2/67] Loss: 3.325 | Mean Reward: 0.532 | KL: 0.297 | Mean Length: 105.5


  3%|█████                                                                                                                                                                   | 2/67 [00:22<12:14, 11.30s/it]

...

 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████     | 65/67 [14:21<00:30, 15.18s/it]

['Q: Honey earned $80 a day. Every day, she spent part of her pay and saved the rest. After 20 days of work, she spent $1360. How much did Honey save in 20 days?\n\nA:  \nShe earned $80/day × 20 days = $1600.  \nShe spent $1360, so she saved $1600 - $1360 = $240.  \n\nThe answer is 240.']
GT Answer:  240
23:30:48 | INFO     | notebook | [66/67] Loss: 0.078 | Mean Reward: 0.424 | KL: 0.079 | Mean Length: 98.0


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 66/67 [14:34<00:14, 14.39s/it]

['Q: Chandler needs to sell 12 rolls of wrapping paper. He has already sold 3 rolls to his grandmother, 4 rolls to his uncle, and 3 rolls to a neighbor. How many more rolls does he need to sell?\n\nA: Chandler has sold a total of 3 + 4 + 3 = 10 rolls. He still needs to sell 12 - 10 = 2 more rolls.  \nThe answer is 2.']
GT Answer:  2
23:30:57 | INFO     | notebook | [67/67] Loss: 0.002 | Mean Reward: 0.376 | KL: 0.002 | Mean Length: 107.5


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [14:43<00:00, 13.19s/it]


In [23]:
lora_model.save_pretrained('ScratchGRPO_200samples/')

### Evaluation

In [24]:

NUM_SUBSET = 100

torch.manual_seed(42)
random_indices = torch.randperm(len(test_dataset))[:NUM_SUBSET]

questions, answers = [], []
for i in random_indices : 
    q, a = build_cot_few_shot_prompt(cot_dataset, test_dataset, int(i), n_shots=2)
    questions.append({'prompt' : q})
    answers.append(a)
    
from torch.utils.data import DataLoader

loader = DataLoader(questions, batch_size = 4, shuffle=False, collate_fn = DataCollator)

from tqdm.auto import tqdm 
decoded_outs = []

# making sure that dropout are disabled
lora_model.eval()

# we don't calculate gradients: we just perform inference
with torch.no_grad() : 
    for batch in tqdm(loader) : 
        # putting the batch on the right GPU
        batch = {k:v.to(device) for k,v in batch.items()}
        out = lora_model.generate(**batch, generation_config=generation_config)
        decoded_outs.append(detokenize_out(batch['input_ids'], out))
        print(decoded_outs[-1][0].replace('<|im_end|>', '').replace('<|endoftext|>', ''))

  4%|██████▋                                                                                                                                                                 | 1/25 [00:08<03:15,  8.14s/it]

Q: Zeke's baseball team has 7 more players than Carlton's. If Carlton's team has 13 players, how many players are there in both teams combined?

A: Carlton's team has 13 players. Zeke's team has $ 13 + 7 = 20 $ players.  
So, the total number of players in both teams combined is $ 13 + 20 = 33 $.  
The answer is 33.


  8%|█████████████▍                                                                                                                                                          | 2/25 [00:20<03:57, 10.35s/it]

Q: A company sold 4000 gallons of milk in jars to Mr. Marcellus' store at the cost of $3.5 per gallon. However, Mr. Marcellus later realized 2/5 of the amount of milk he purchased had passed the expiry date and could not be sold. He returned the sour milk to the company and ordered a refund. Calculate how much he got in refunds.

A:  
He sold 4000 gallons of milk for $3.5/gallon × 4000 = $14,000.  
Out of this, 2/5 of the milk (which is 1600 gallons) was expired and not sold.  
He returned the sour milk to the company and received a refund for the cost of the expired milk.  
The refund amount is $1600 × $3.5 = $5,600.  

The answer is 5,600.


 12%|████████████████████▏                                                                                                                                                   | 3/25 [00:36<04:46, 13.02s/it]

...

Q: Rose went to the store on Monday and bought 4 cakes. Tuesday she went to a different store and bought three times that number of cakes. On Wednesday she went to another store and bought 5 times the number of cakes she did on Tuesday. How many cakes did she buy after all three days?

A:  
- Monday: 4 cakes  
- Tuesday: 3 times Monday’s purchase = $3 \times 4 = 12$ cakes  
- Wednesday: 5 times Tuesday’s purchase = $5 \times 12 = 60$ cakes  

Total cakes bought: $4 + 12 + 60 = 76$  

The answer is 76.


 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎      | 24/25 [04:27<00:10, 10.81s/it]

Q: In a company of 50 employees, 20% of the employees are management. Out of this 20%, only 30% oversee the entire company. How many employees oversee the company?

A:  
- 20% of 50 employees is 10 management employees.  
- 30% of 10 management employees is 3 employees.  

The answer is 3.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [04:39<00:00, 11.18s/it]

Q: A class of 200 students is split into 3 groups such that 2 of them are equal in number and the last one (which is the smallest) is 10 less than each of the other groups. How many students are in this (smallest) group?

A: Let the smallest group be $ x $ students. The two groups equal in size are $ x + 10 $, and the third group is $ x + 10 + 10 = x + 20 $. These three groups add up to 200, so:
$$ 200 = (x + 10) + (x + 10) + (x + 20) $$
$$ 200 = 3x + 40 $$
$$ 200 - 40 = 3x $$
$$ 160 = 3x $$
$$ x = \frac{160}{3} $$

**Answer:** There are $ \boxed{33} $ students in the smallest group.


In [25]:
# Flatten list of list into list
decoded_outs = sum(decoded_outs, [])
pred_answers = [extract_answer_from_model(gen) for gen in decoded_outs]
accuracy = sum([(p==gt) for p,gt in zip(pred_answers, answers)]) / len(answers)
print(f"(Scratch GRPO Model) Accuracy on Test Subsample: {int(accuracy*100)}%")

(Scratch GRPO Model) Accuracy on Test Subsample: 38%


In [26]:
invalids = [i for i in pred_answers if 'invalid' in i]
print(f'% of Invalid Answers: {(len(invalids)/len(decoded_outs)) * 100:.2f}')

% of Invalid Answers: 16.00


## TRL 

Now we will use a dedicated library, `trl` to see how it differs from our implementation. Notably, it is optimized for being:
- Much faster
- Consumes less GPU memory
- Contains all *training tricks* to have a stable training

We start by reloading the model to avoid reusing a pre-trained model

In [13]:
NUM_TRAIN = 200

torch.manual_seed(42)
random_indices = torch.randperm(len(test_dataset))

train = []
for i in random_indices[:NUM_TRAIN] : 
    q, a = build_cot_few_shot_prompt(cot_dataset, train_dataset, int(i), n_shots=2)
    train.append({'prompt' : [
        {
            'role' : 'user',
            # TRL use default chat templated. We need to add the /no_thing to disable thinking in Qwen3 models
            'content' : q + '/no_think' 
        }
    ], 'answer' : a})


# Eval dataset. We wont use it here because it takes too much time to evaluate
eval_d = []
for i in random_indices[NUM_TRAIN:NUM_TRAIN+20] : 
    q, a = build_cot_few_shot_prompt(cot_dataset, train_dataset, int(i), n_shots=2)
    eval_d.append({'prompt' : [
        {
            'role' : 'user',
            'content' : q + '/no_think'
        }
    ], 'answer' : a})
    

In [14]:
import argparse
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import GRPOConfig, GRPOTrainer

In [15]:
def format_reward_fn( completions, **kwargs):
    pattern = r"The answer is [\d]+"
    completions = [c[0]['content'] for c in completions]
    matches = [re.search(pattern, content) for content in completions]
    return [0.5 if match else 0.0 for match in matches]

def correct_reward_fn(completions, answer, **kwargs) :
    # print(completions)
    completions = [c[0]['content'] for c in completions]
    preds = [extract_answer_from_model(i) for i in completions]
    return  [1  if preds[i] == answer[i] else 0 for i in range(len(completions))]


In [16]:

training_args = GRPOConfig(
    output_dir='./logs/',
    run_name=f'GRPO-GSM8K',
    learning_rate=2e-5, #Learning rate
    logging_steps=1,
    fp16=False, 
    bf16=True, # We train the model in bf16 precision --> way faster
    log_completions=True, # Will log stats about completions
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    save_steps=100,
    log_on_each_node=True,
    log_level='info', 
    logging_dir='./logs/',
    do_train=True,
    disable_dropout=True,
    report_to='wandb',
    eval_strategy='steps',
    eval_steps = 500
    
    # Batch and other parameters
    num_generations=4, # Group size: number of generations per prompt
    per_device_train_batch_size=4, # Rollout size 
    num_iterations=2, # Number of off-policy iterations

    ### Sampling parameters
    top_p=0.95, 
    temperature=1.0,
    max_completion_length=300,

    # PPO-Loss hyperparameters
    beta=0.05, # KL-Div coefficient  
    epsilon = 0.2, # epsilon Clip value
    max_grad_norm=1.0, # clip gradient if too large

    # vLLM parameters. We dont use it here, it doesnt work on collab
    # but should be used if access to local
    # use_vllm=True,
    # vllm_device='auto',
)


`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [17]:
# Some utilities here
model.config.pad_token_id = tokenizer.pad_token_id
tokenizer.padding_side = 'left'

In [ ]:
trainer = GRPOTrainer(
    
    model=model,
    processing_class=tokenizer,
    # Reward functions
    reward_funcs=[format_reward_fn, correct_reward_fn],
    args=training_args,
    train_dataset=train,
    # PEFT config
    peft_config=peft_config,
    # We put it even if we won't evaluate to force the log of training values
    eval_dataset=eval_d
)

trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
***** Running training *****
  Num examples = 200
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 400
  Number of trainable parameters = 10,092,544
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
wandb: Currently logged in as: gaspardmichel (deezer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing `generation_config` together with generation-related arguments=({'disable_compile'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


Step,Training Loss,Validation Loss


In [ ]:
trainer.save_model('GRPO_500samples/')

## Evaluation

In [74]:

NUM_SUBSET = 100

torch.manual_seed(42)
random_indices = torch.randperm(len(test_dataset))[:NUM_SUBSET]

questions, answers = [], []
for i in random_indices : 
    q, a = build_cot_few_shot_prompt(cot_dataset, test_dataset, int(i), n_shots=2)
    questions.append({'prompt' : q +'/no_think'})
    answers.append(a)
from torch.utils.data import DataLoader

loader = DataLoader(questions, batch_size = 4, shuffle=False, collate_fn = DataCollator)

### Load from save

In [75]:
from peft import PeftModel
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16,
    device_map="cpu",
)
device = torch.device("cuda:0")

rl_trained_model = PeftModel.from_pretrained(ref_model, 'GRPO_500samples/').to(device)

Loading weights: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 311/311 [00:00<00:00, 1306.36it/s, Materializing param=model.norm.weight]
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [77]:


from tqdm.auto import tqdm 
decoded_outs = []

# making sure that dropout are disabled
rl_trained_model.eval()

# we don't calculate gradients: we just perform inference
with torch.no_grad() : 
    for batch in tqdm(loader) : 
        # putting the batch on the right GPU
        batch = {k:v.to(device) for k,v in batch.items()}
        out = rl_trained_model.generate(**batch, generation_config=generation_config)
        decoded_outs.append(detokenize_out(batch['input_ids'], out))
        print(decoded_outs[-1][0].replace('<|endoftext|>',''))

  4%|██████▋                                                                                                                                                                 | 1/25 [00:09<03:49,  9.55s/it]

Q: Zeke's baseball team has 7 more players than Carlton's. If Carlton's team has 13 players, how many players are there in both teams combined?

A: Zeke's team has 13 + 7 = 20 players.  
So, in total, there are 20 (Zeke's team) + 13 (Carlton's team) = 33 players in both teams combined. The answer is 33.<|im_end|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

  8%|█████████████▍                                                                                                                                                          | 2/25 [00:20<03:55, 10.24s/it]

Q: A company sold 4000 gallons of milk in jars to Mr. Marcellus' store at the cost of $3.5 per gallon. However, Mr. Marcellus later realized 2/5 of the amount of milk he purchased had passed the expiry date and could not be sold. He returned the sour milk to the company and ordered a refund. Calculate how much he got in refunds.

A:  
Mr. Marcellus sold 4000 gallons at $3.5 per gallon, so the total cost was $3.5 * 4000 = 14,000.  
Of those 4000 gallons, 2/5 were returned due to expiry, so 2/5 of 4000 is 1600 gallons.  
The refund amount is 1600 * $3.5 = $5,600.  

The answer is 5600.<|im_end|>


 12%|████████████████████▏                                                                                                                                                   | 3/25 [00:30<03:49, 10.42s/it]

...

Q: Rose went to the store on Monday and bought 4 cakes. Tuesday she went to a different store and bought three times that number of cakes. On Wednesday she went to another store and bought 5 times the number of cakes she did on Tuesday. How many cakes did she buy after all three days?

A:  
On Monday, Rose bought 4 cakes.  
On Tuesday, she bought 3 × 4 = 12 cakes.  
On Wednesday, she bought 5 × 12 = 60 cakes.  

So, Rose bought a total of 4 + 12 + 60 = 76 cakes.  

The answer is 76.<|im_end|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endofte

 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎      | 24/25 [04:34<00:11, 11.99s/it]

Q: In a company of 50 employees, 20% of the employees are management. Out of this 20%, only 30% oversee the entire company. How many employees oversee the company?

A: The number of employees in the management team is 20% of 50, which equals 10. Out of these 10, only 30% oversee the entire company, so 10 * 30% = 3 employees oversee the company.

The answer is 3.<|im_end|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [04:46<00:00, 11.48s/it]

Q: A class of 200 students is split into 3 groups such that 2 of them are equal in number and the last one (which is the smallest) is 10 less than each of the other groups. How many students are in this (smallest) group?

A: Let the number of students in each of the two equal groups be $ x $. Then the smallest group has $ x - 10 $ students. There are 2 groups with $ x $ students each, so the total number of students is $ 2x + (x - 10) = 3x - 10 $. This is equal to the total number of students in the class, which is 200. 

So, $ 3x - 10 = 200 $.

Solving for $ x $:

$$
3x = 210 \quad \Rightarrow \quad x = 70
$$

Thus, the smallest group has $ 70 - 10 = 60 $ students.

The answer is 60.<|im_end|>


In [78]:
# Flatten list of list into list
decoded_outs = sum(decoded_outs, [])
pred_answers = [extract_answer_from_model(gen) for gen in decoded_outs]
accuracy = sum([(p==gt) for p,gt in zip(pred_answers, answers)]) / len(answers)
print(f"(TRL GRPO Model) Accuracy on Test Subsample: {int(accuracy*100)}%")

(SFT Model) Accuracy on Test Subsample: 49%


**Nice improvement over SFT and Base model!**

In [79]:
# Very high percentage of invalid answers
invalids = [i for i in pred_answers if 'invalid' in i]
print(f'% of Invalid Answers: {(len(invalids)/len(decoded_outs)) * 100:.2f}')

% of Invalid Answers: 9.00


High percentage of invalid answers means that something is wrong
- Can be too small `max_token` : completion is being truncated before ending
- A solution can be to **penalize the model if it doesn't generate an `EOS` token**

# Evaluation Results

|                         | Solve Rate (%) |
|-------------------------|:--------------:|
| Base Model (Qwen3-0.6B) |       31       |
| SFT Model               |        6       |
| GRPO Model (Scratch)         |       38       |
| GRPO Model (TRL)         |       **49**       |

We improved by 18 accuracy points with as little as 200 data points!

Cons:

- GRPO training is quite slow, and memory intensive
- Can be speed up with vLLM but does not run on Collab